In [1]:
import boto3

# Crear cliente de EC2
ec2_client = boto3.client('ec2', region_name='us-east-1')

# Listar instancias EC2
try:
    response = ec2_client.describe_instances()
    instances = response.get('Reservations', [])
    print("Instancias EC2:")
    for reservation in instances:
        for instance in reservation.get('Instances', []):
            print(f"- ID: {instance['InstanceId']}, Estado: {instance['State']['Name']}")
except Exception as e:
    print("Error al listar instancias EC2:", str(e))

# Crear cliente de S3
s3_client = boto3.client('s3', region_name='us-east-1')

# Listar buckets S3
try:
    response = s3_client.list_buckets()
    buckets = response.get('Buckets', [])
    print("Buckets S3:")
    for bucket in buckets:
        print(f"- {bucket['Name']}")
except Exception as e:
    print("Error al listar buckets S3:", str(e))

Instancias EC2:
- ID: i-00814784d7c10c4ef, Estado: running
- ID: i-033bb8b829a8a3e6d, Estado: running
Buckets S3:
- prueba010206


## Crear una instancia EC2, ejecutarla, pararla y eliminarla

In [ ]:
# Crear una instancia EC2
try:
    ec2_response = ec2_client.run_instances(
        ImageId='ami-053a45fff0a704a47',  # Reemplaza con el ID de la AMI que desees usar
        InstanceType='t2.micro',  # Tipo de instancia t2.micro
        KeyName='prueba',  # Nombre de la llave para acceder a la instancia(Creado otro dia en clase)
        MinCount=1,
        MaxCount=1,
        Placement={
            'AvailabilityZone': 'us-east-1d'  # Zona de disponibilidad
        },
        TagSpecifications=[
            {
                'ResourceType': 'instance',
                'Tags': [
                    {
                        'Key': 'Name',
                        'Value': 'tarea_almacenamiento'
                    }
                ]
            }
        ]
    )
    instance_id = ec2_response['Instances'][0]['InstanceId']
    print(f"Instancia EC2 creada con ID: {instance_id}")
except Exception as e:
    print("Error al crear la instancia EC2:", str(e))

Instancia EC2 creada con ID: i-0ea578e385c946d04


In [24]:
# Ejecutar la instancia EC2
try:
    ec2_client.start_instances(InstanceIds=[instance_id])
    print(f"Instancia EC2 {instance_id} iniciada")
except Exception as e:
    print("Error al iniciar la instancia EC2:", str(e))

Instancia EC2 i-0ea578e385c946d04 iniciada


In [12]:
# Parar la instancia EC2
try:
    ec2_client.stop_instances(InstanceIds=[instance_id])
    print(f"Instancia EC2 {instance_id} parada")
except Exception as e:
    print("Error al parar la instancia EC2:", str(e))

Instancia EC2 i-0f5a04458c56f58b3 parada


In [13]:
# Eliminar la instancia EC2
try:
    ec2_client.terminate_instances(InstanceIds=[instance_id])
    print(f"Instancia EC2 {instance_id} eliminada")
except Exception as e:
    print("Error al eliminar la instancia EC2:", str(e))

Instancia EC2 i-0f5a04458c56f58b3 eliminada


## Crear un EBS y asociarlo al EC2 anterior y añadir un archivo

In [ ]:
import time
import paramiko

# Crear un volumen EBS
try:
    volume_response = ec2_client.create_volume(
        AvailabilityZone='us-east-1d',  # Zona de disponibilidad de la instancia EC2
        Size=1,  # Tamaño del volumen en GB
        VolumeType='gp2',  # Tipo de volumen
        TagSpecifications=[
            {
                'ResourceType': 'volume',
                'Tags': [
                    {
                        'Key': 'Name',
                        'Value': 'tarea_almacenamiento_ebs'
                        },
                        {
                            'Key': 'AvailabilityZone',
                            'Value': 'us-east-1d'
                    }
                ]
            }
        ]
    )
    volume_id = volume_response['VolumeId']
    print(f"Volumen EBS creado con ID: {volume_id}")
except Exception as e:
    print("Error al crear el volumen EBS:", str(e))

# Esperar a que el volumen esté disponible
while True:
    volume_status = ec2_client.describe_volumes(VolumeIds=[volume_id])
    if volume_status['Volumes'][0]['State'] == 'available':
        break
    time.sleep(5)

Volumen EBS creado con ID: vol-03a8f2727436c4782


In [25]:
# Asociar el volumen EBS a la instancia EC2
try:
    ec2_client.attach_volume(
        VolumeId=volume_id,
        InstanceId=instance_id,
        Device='/dev/sdf'  # Dispositivo en el que se montará el volumen
    )
    print(f"Volumen EBS {volume_id} asociado a la instancia EC2 {instance_id}")
except Exception as e:
    print("Error al asociar el volumen EBS a la instancia EC2:", str(e))

Error al asociar el volumen EBS a la instancia EC2: An error occurred (InvalidVolume.ZoneMismatch) when calling the AttachVolume operation: The volume 'vol-03a8f2727436c4782' is not in the same availability zone as instance 'i-0ea578e385c946d04'


In [ ]:

# Conectar a la instancia EC2 y montar el volumen
# Este paso requiere acceso SSH a la instancia EC2
# Puedes usar paramiko o cualquier otra librería para ejecutar comandos SSH en la instancia EC2

# Configurar la conexión SSH
key = paramiko.RSAKey(filename='prueba.pem')  # Reemplaza con la ruta a tu archivo de clave privada
ssh_client = paramiko.SSHClient()
ssh_client.set_missing_host_key_policy(paramiko.AutoAddPolicy())

try:
    ssh_client.connect(hostname='ec2-44-202-245-215.compute-1.amazonaws.com', username='ec2-user', pkey=key)  # Reemplaza con la dirección pública de tu instancia EC2

    # Montar el volumen EBS
    commands = [
        'sudo mkfs -t ext4 /dev/xvdf',  # Formatear el volumen
        'sudo mkdir /mnt/ebs',  # Crear un directorio para montar el volumen
        'sudo mount /dev/xvdf /mnt/ebs',  # Montar el volumen
        'echo "Hello, EBS!" | sudo tee /mnt/ebs/hello.txt'  # Crear un archivo en el volumen montado
    ]
    for command in commands:
        stdin, stdout, stderr = ssh_client.exec_command(command)
        print(stdout.read().decode())
        print(stderr.read().decode())

    print("Archivo creado en el volumen EBS montado")
except Exception as e:
    print("Error al conectar a la instancia EC2 y montar el volumen EBS:", str(e))
finally:
    ssh_client.close()

## Crear un EFS, montarlo y añadir un archivo

## Crear un S3 Estándar, crear un cubo y añadir varias carpetas con un objeto que sea un archivo csv con varios datos para trabajar con él a posteriori y obtener le objeto

## Crear S3 Estándar - Acceso poco frecuente, crear un cubo y añadir un objeto y obtener le objeto

## Crear S3 Intelligent-Tiering, crear un cubo y añadir un objeto y obtener le objeto

## Crear S3 Glacier, crear un cubo y añadir un objeto y obtener le objeto

## Crear S3 Glacier Deep Archive, crear un cubo y añadir un objeto y obtener le objeto

## Hablitar el control de versiones de S3 mediante comandos y mostrar un ejemplo de un objeto modificado y mostrar dos versiones

## Realizar 3 consultas diferentes sobre el objeto .csv del S3 usando AWS Athena